In [56]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.express as px
import plotly.express as px
import numpy as np
url= os.path.join(os.getcwd(), "data", "base_transacciones_actualizada.xlsx")
data = pd.read_excel(url, sheet_name="Sheet 1")
data=data[['Transaccion','Monto','Duración (minutos)']]

In [55]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2
mean = data.mean().values
cov_matrix = np.cov(data.values, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)
data['Mahalanobis'] = data.apply(lambda row: mahalanobis(row.values, mean, inv_cov_matrix), axis=1)
p_value = 0.90
chi2_critical = chi2.ppf(p_value, df=data.shape[1] - 1)
data['Outlier'] = data['Mahalanobis'] > np.sqrt(chi2_critical)
print(data)
fig = px.scatter(
    data,
    x='Transaccion',
    y='Monto',
    color='Outlier',
    title='Distancia de Mahalanobis y Valores Atípicos',
    labels={'Outlier': 'Es Atípico'},
    template='plotly_white'
)
fig.show()

       Transaccion  Monto  Duración (minutos)  Mahalanobis  Outlier
0      3656.478592   2468                  19     1.483022    False
1      5621.775554   5372                   8     1.357576    False
2      5800.874665   9389                  38     1.658928    False
3      3611.107588   1351                  37     1.677519    False
4      4285.643141   8925                  19     1.576576    False
...            ...    ...                 ...          ...      ...
24995  4365.992777   4451                  14     1.079897    False
24996  6023.046829   7739                  11     1.601510    False
24997  5642.079885    696                  43     1.738815    False
24998  4898.526567   1351                  15     1.586674    False
24999  5122.344553   5919                  32     0.313398    False

[25000 rows x 5 columns]


In [40]:
len(list(np.where(data.Outlier==True))[0])

320

In [62]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
import plotly.express as px
from scipy.stats import norm

url= os.path.join(os.getcwd(), "data", "base_transacciones_actualizada.xlsx")
data = pd.read_excel(url, sheet_name="Sheet 1")
data=data[['Transaccion','Monto','Duración (minutos)']]

mean = data.mean().values  # Vector de medias
cov_matrix = np.cov(data.values, rowvar=False)  # Matriz de covarianza
inv_cov_matrix = np.linalg.inv(cov_matrix)  # Matriz inversa de covarianza
def manual_mahalanobis(row, mean, inv_cov_matrix):
    diff = row - mean  # Diferencia entre el punto y la media
    dist_squared = np.dot(np.dot(diff.T, inv_cov_matrix), diff)  # (x-µ)^T Σ^-1 (x-µ)
    return np.sqrt(dist_squared)  # Raíz cuadrada para obtener la distancia
data['Mahalanobis'] = data.apply(lambda row: manual_mahalanobis(row.values, mean, inv_cov_matrix), axis=1)

# Calcular la media y la desviación estándar
mean_transaccion = data['Transaccion'].mean()
std_transaccion = data['Transaccion'].std()
#outliers_transaccion = data[
#    (data['Transaccion'] > mean_transaccion + 3 * std_transaccion) |
#    (data['Transaccion'] < mean_transaccion - 3 * std_transaccion)
#]
#data['Outlier'] = (
#    (data['Transaccion'] > mean_transaccion + 3 * std_transaccion) |
#    (data['Transaccion'] < mean_transaccion - 3 * std_transaccion)
#)


data['Confiabilidad'] = np.abs(data['Transaccion'] - mean_transaccion) / std_transaccion
data['p-valor'] = 2 * (1 - norm.cdf(data['Confiabilidad']))
data['Outlier'] = data['p-valor'] < 0.10


#p_value = 0.95
#chi2_critical = chi2.ppf(p_value, df=data.shape[1])  # Grados de libertad = número de columnas
#data['Outlier'] = data['Mahalanobis'] > np.sqrt(chi2_critical)




print(data)
fig = px.scatter(
    data,
    x='Transaccion',
    y='Monto',
    color='Outlier',
    title='Distancia de Mahalanobis y Valores Atípicos',
    labels={'Outlier': 'Es Atípico'},
    template='plotly_white'
)
fig.show()

       Transaccion  Monto  Duración (minutos)  Mahalanobis  Confiabilidad  \
0      3656.478592   2468                  19     1.483022       0.957806   
1      5621.775554   5372                   8     1.357576       0.406442   
2      5800.874665   9389                  38     1.658928       0.530767   
3      3611.107588   1351                  37     1.677519       0.989301   
4      4285.643141   8925                  19     1.576576       0.521060   
...            ...    ...                 ...          ...            ...   
24995  4365.992777   4451                  14     1.079897       0.465284   
24996  6023.046829   7739                  11     1.601510       0.684992   
24997  5642.079885    696                  43     1.738815       0.420536   
24998  4898.526567   1351                  15     1.586674       0.095615   
24999  5122.344553   5919                  32     0.313398       0.059752   

        p-valor  Outlier  
0      0.338160    False  
1      0.684418    Fa

In [63]:
len(list(np.where(data.Outlier==True))[0])

518